In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import openai
from openai import OpenAI
import time
from tqdm import tqdm

from diskcache import Cache
cache = Cache("/shared_data0/llm_cachedir")


### OpenAI Querying Functions

In [2]:
@cache.memoize()
def query_openai(prompt, model="gpt-4o"):
    with open("../API_KEY.txt", "r") as file:
        api_key = file.read()
    client = OpenAI(api_key=api_key)

    num_tries = 0
    for i in range(3):
        try:
            translation = client.chat.completions.create(
                messages=[{
                    "role": "user",
                    "content": prompt,
                }],
                model=model,
            )
            return translation.choices[0].message.content
        except Exception as e:
            num_tries += 1
            print("Try {}; Error: {}".format(str(num_tries), str(e)))     
            time.sleep(3)
    return "ERROR"

### Load Emotion Data

In [3]:
emotion_data =  load_dataset("BrachioLab/emotion")
emotion_data = emotion_data['train'].to_pandas()
emotion_data = emotion_data.sample(20, random_state=11).reset_index(drop=True)

class EmotionExample:
    def __init__(self, text, ground_truth, llm_label, llm_explanation):
        self.text = text
        self.ground_truth = ground_truth
        self.llm_label = llm_label
        self.llm_explanation = llm_explanation
        self.claims = []
        self.relevant_claims = []
        self.alignment_scores = []

In [4]:
emotion_labels = {
    0: "admiration",
    1: "amusement",
    2: "anger",
    3: "annoyance",
    4: "approval",
    5: "caring",
    6: "confusion",
    7: "curiosity",
    8: "desire",
    9: "disappointment",
    10: "disapproval",
    11: "disgust",
    12: "embarrassment",
    13: "excitement",
    14: "fear",
    15: "gratitude",
    16: "grief",
    17: "joy",
    18: "love",
    19: "nervousness",
    20: "optimism",
    21: "pride",
    22: "realization",
    23: "relief",
    24: "remorse",
    25: "sadness",
    26: "surprise",
    27: "neutral"
}

### Stage 0: Get LLM Explanations


In [5]:
explanation_prompt = """What is the emotion of the following text? Here are the possible labels you could use:
admiration
amusement
anger
annoyance
approval
caring
confusion
curiosity
desire
disappointment
disapproval
disgust
embarrassment
excitement
fear
gratitude
grief
joy
love
nervousness
optimism
pride
realization
relief
remorse
sadness
surprise
neutral

In addition, provide a paragraph explaining why you gave the text that classification label. Your response should be 2 lines, formatted as follows:
Label: <label>
Explanation: <explanation>

Here is the following text.
Text: {}
"""
def get_llm_generated_answer(text: str):
    prompt = explanation_prompt.format(text)
    response = query_openai(prompt)
    if response == "ERROR":
        print("Error in querying OpenAI API")
        return None
    response_split = [e for e in response.split("\n") if (e != '' and e.split()[0] in ['Label:', 'Explanation:'])]
    llm_label = response_split[0].split("Label: ")[1].strip()
    explanation = response_split[1].split("Explanation: ")[1].strip()
    return llm_label, explanation

emotion_examples = []
for idx,row in tqdm(emotion_data.iterrows()):
    llm_label, explanation = get_llm_generated_answer(row['text'])
    if llm_label is None:
        continue
    emotion_examples.append(EmotionExample(
        text=row['text'],
        ground_truth=emotion_labels[row['labels'][0]],
        llm_label=llm_label,
        llm_explanation=explanation
    ))

20it [00:19,  1.02it/s] 


In [20]:
[emotion_examples[i].llm_label for i in range(len(emotion_examples))]

['amusement',
 'amusement',
 'joy',
 'approval',
 'amusement',
 'gratitude',
 'understanding',
 'disapproval',
 'amusement',
 'curiosity',
 'gratitude',
 'approval',
 'disapproval',
 'curiosity',
 'embarrassment',
 'caring',
 'curiosity',
 'annoyance',
 'surprise',
 'urgency']

In [7]:
emotion_examples[0].ground_truth

'neutral'

In [8]:
emotion_examples[0].text

'Creepy Yoda voicepack tbh'

In [9]:
emotion_examples[0].llm_label

'amusement'

In [10]:
emotion_examples[0].llm_explanation

'The use of "Creepy Yoda voicepack tbh" suggests a tongue-in-cheek or playful reaction, as "tbh" (to be honest) paired with "creepy" implies a light-hearted observation rather than fear or disgust.'

### Stage 1: Atomic claim extraction

In [11]:
claim_prompt = """
You will be given a paragraph that explains why a emotion was attributed to an utterance. Your task is to decompose this explanation into individual claims that are:

Atomic: Each claim should express only one clear idea or judgment.
Standalone: Each claim should be self-contained and understandable without needing to refer back to the paragraph.
Faithful: The claims must preserve the original meaning, nuance, and tone. Do not omit hedging language (e.g., "seems to," "somewhat," "lacks overt markers") or subjective phrasing if present.

Format your output as a list of claims separated by new lines. Do not include any additional text or explanations.

Here is an example of how to format your output:

INPUT: The use of "creepy" and "tbh" suggests a negative reaction or discomfort, likely indicating the speaker finds the voicepack irritating or unsettling.

OUTPUT:

The use of the word "creepy" suggests a negative reaction or discomfort.
The word "tbh" likely indicates that the speaker finds the voicepack irritating or unsettling.

Now decompose the following pararaph into atomic, standalone claims:
INPUT: {}
"""


def isolate_individual_features(explanation: str):
    prompt = claim_prompt.format(explanation)
    response = query_openai(prompt)
    if response == "ERROR":
        print("Error in querying OpenAI API")
        return None
    response = response.replace("OUTPUT:", "").strip()
    claims = response.split("\n")
    return claims

for example in emotion_examples:
    claims = isolate_individual_features(example.llm_explanation)
    if claims is None:
        continue
    example.claims = [claim.strip() for claim in claims]

In [12]:
emotion_examples[0].claims

['The phrase "Creepy Yoda voicepack tbh" suggests a tongue-in-cheek or playful reaction.',
 'The word "tbh" (to be honest) paired with "creepy" implies a light-hearted observation rather than fear or disgust.']

In [13]:
relevance_prompt = """You will be given a text, its emotion label, and a claim that may or may not be relevant to an explanation of the emotion label. Your task is to decide whether the claim is relevant to explaining the emotion label for this specific text.

A claim is relevant if and only if:
(1) It is supported by the content of the utterance (i.e., it does not hallucinate or speculate beyond what is said).
(2) It helps explain why the utterance received the given emotion label (i.e., it directly relates to tone, phrasing, or other aspects relevant to the label).

Return your answer as:
Relevance: <Yes/No>
Reasoning: <A brief explanation of your judgment, pointing to specific support or lack thereof>

Now, determine whether the following claim is relevant to the given text and emotion label:
Text: {}
Emotion Label: {}
Claim: {}"""

In [14]:
prompt = relevance_prompt.format(emotion_examples[0].text, emotion_examples[0].llm_label, emotion_examples[0].claims[0])
response = query_openai(prompt)
print("Text:", emotion_examples[0].text)
print("Emotion Label:", emotion_examples[0].llm_label)
print("Claim:", emotion_examples[0].claims[0])
print(response)

Text: Creepy Yoda voicepack tbh
Emotion Label: amusement
Claim: The phrase "Creepy Yoda voicepack tbh" suggests a tongue-in-cheek or playful reaction.
Relevance: Yes  
Reasoning: The claim directly relates to and is supported by the content of the text. The use of "Creepy Yoda voicepack tbh" can indeed suggest a tongue-in-cheek or playful reaction, which aligns well with the emotion label of amusement. The choice of words implies a light-hearted or humorous tone, satisfying both conditions for relevance.


In [16]:
prompt = relevance_prompt.format(emotion_examples[7].text, emotion_examples[7].llm_label, emotion_examples[7].claims[0])
response = query_openai(prompt)
print("Text:", emotion_examples[7].text)
print("Emotion Label:", emotion_examples[7].llm_label)
print("Claim:", emotion_examples[7].claims[0])
print(response)

Text: This feels more like r/fellowkids to me. This is nothing compared to some of the more dystopian stuff the government has pushed out to be fair.
Emotion Label: disapproval
Claim: The text expresses a sense of disapproval or criticism.
Relevance: Yes  
Reasoning: The claim is relevant because it is supported by the content of the text. The text expresses criticism by comparing the subject to "r/fellowkids," which is often used to criticize attempts at trying too hard to appeal to younger audiences. Additionally, stating that this is "nothing compared to some of the more dystopian stuff the government has pushed out" implies a critical or disapproving tone about the subject, which aligns with the emotion label of disapproval.


### Stage 2: Distill relevant claims

In [23]:
relevance_prompt = """You will be given a text, its emotion label, and a claim that may or may not be relevant to an explanation of the emotion label. Your task is to decide whether the claim is relevant to explaining the emotion label for this specific text.

A claim is relevant if and only if:
(1) It is supported by the content of the text (i.e., it does not hallucinate or speculate beyond what is said).
(2) It helps explain why the text received the given emotion label (i.e., it directly relates to tone, phrasing, or other aspects relevant to the label).

Return your answer as:
Relevance: <Yes/No>
Reasoning: <A brief explanation of your judgment, pointing to specific support or lack thereof>

Here are some examples:

[Example 1]
Text: Creepy Yoda voicepack tbh
Emotion Label: disapproval
Claim: The use of the word "creepy" paired with "Yoda voicepack" suggests a playful tone.
Relevance: No  
Reasoning: The claim is not relevant because it does not contribute to the emotion label of disapproval. 

[Example 2]
Text: Motor Kombat was awesome. Goofy as hell, but awesome.
Emotion Label: amusement
Claim: There is an entertaining sentiment typically associated with amusement.
Relevance: Yes  
Reasoning: The claim is relevant because it accurately captures the tone of the text. The description of "Motor Kombat" as "awesome" and "goofy as hell" suggests an enjoyable and entertaining experience, which aligns with the entertaining sentiment associated with amusement. The text's phrasing supports the emotion label of amusement.

[Example 3]
Text: And they lived happily ever after.
Emotion Label: joy
Claim: The phrase typically signifies a positive and fulfilling conclusion to a story.
Relevance: Yes
Reasoning: The claim is relevant because the phrase "And they lived happily ever after" is commonly associated with fairy tales and stories that end on a positive, satisfying note. This association supports the emotion label of amusement by indicating a light-hearted and cheerful conclusion, which is often found in amusing and entertaining stories. The claim does not speculate beyond what is expressed in the text, as the phrase itself directly suggests a positive ending, aligning with feelings of amusement.

Now, determine whether the following claim is relevant to the given text and emotion label:
Text: {}
Emotion Label: {}
Claim: {}
"""

def is_claim_relevant(text: str, rating: str, claim: str):
    prompt = relevance_prompt.format(text, rating, claim)
    response = query_openai(prompt)
    if response == "ERROR":
        print("Error in querying OpenAI API")
        return None
    response = response.replace("Relevance:", "").strip()
    response = response.split("\n")
    relevance = response[0].strip()
    reasoning = response[1].replace("Reasoning:", "").strip()
    return relevance, reasoning


def distill_relevant_features(example: EmotionExample):
    relevant_claims = []
    for claim in tqdm(example.claims):
        relevance, reasoning = is_claim_relevant(example.text, example.llm_label, claim)
        if relevance is None:
            continue
        if relevance == "Yes":
            relevant_claims.append(claim)
    return relevant_claims

for example in emotion_examples:
    relevant_claims = distill_relevant_features(example)
    example.relevant_claims = relevant_claims

100%|██████████| 3/3 [00:00<00:00, 959.50it/s]


In [22]:
emotion_examples[0].relevant_claims

['The phrase "Creepy Yoda voicepack tbh" suggests a tongue-in-cheek or playful reaction.',
 'The word "tbh" (to be honest) paired with "creepy" implies a light-hearted observation rather than fear or disgust.']

### Stage 3: Calculate alignment scores


In [24]:
alignment_prompt = """You will be given a text, its emotion label, and a claim that relates to why that label was given. 

Your task is as follows:
1. On a -2 to 2 scale, rate the valence of what is conveyed in the claim. 
2. On a -2 to 2 scale, rate the arousal of what is conveyed in the claim. 

Valence is the degree of positivity or negativity expressed in a sentence. It ranges from negative (sadness, displeasure) to positive (happiness, pleasure). -2 = very negative valence and 2 = very positive valence.
Arousal is the level of intensity or energy conveyed by the emotion in a sentence. It ranges from low (calm, tired) to high (alarmed, astonished). -2 = very low arousal and 2 = very high arousal.


Return your answer as:
Valence Rating: <rating>
Arousal Rating: <rating>
Reasoning: <A brief explanation of why you gave the valence and arousal ratings that you did.>

Here are some examples:

[Example 1]
Text: And they lived happily ever after.
Emotion Label: joy
Claim: The phrase "And they lived happily ever after" conveys a sense of joy.
Valence Rating: 2
Arousal Rating: -1
Reasoning: The claim emphasizes the positive emotional resolution implied by the phrase “And they lived happily ever after,” which is strongly associated with joy and satisfaction. However, while the valence is very high due to the happy sentiment, the arousal is relatively low—it suggests contentment and peace rather than excitement or high energy.

[Example 2]
Text: Motor Kombat was awesome. Goofy as hell, but awesome.
Emotion Label: amusement
Claim: There is an entertaining sentiment typically associated with amusement.
Valence Rating: 2
Arousal Rating: 1.5
Reasoning: The claim highlights a strongly positive, fun sentiment (“awesome,” “amusement”) indicating high valence. The enthusiastic tone also suggests a relatively high energy or arousal level.

[Example 3]
Text: This feels more like r/fellowkids to me. This is nothing compared to some of the more dystopian stuff the government has pushed out to be fair.
Emotion Label: disapproval
Claim: The feeling of the content being forced or out of touch aligns with a sense of disapproval.
Valence Rating: -1
Arousal Rating: 0.5
Reasoning: The claim highlights a negative evaluation—feeling that something is "forced or out of touch"—which supports a moderately negative valence. The arousal is slightly above neutral because the tone implies a level of irritation or critical engagement, but it isn’t highly emotional or intense.

Now, determine the valence and arousal ratings for the following claim:
Text: {}
Emotion Label: {}
Claim: {}
"""

def calculate_expert_alignment_score(text: str, label: str, claim: str):
    prompt = alignment_prompt.format(text, label, claim)
    response = query_openai(prompt)
    if response == "ERROR":
        print("Error in querying OpenAI API")
        return None
    response = [e for e in response.strip().split("\n") if e != ""]
    valence_rating = float(response[0].replace("Valence Rating:", "").strip())
    arousal_rating = float(response[1].replace("Arousal Rating:", "").strip())
    alignment_score = max(abs(valence_rating), abs(arousal_rating))
    reasoning = response[2].replace("Reasoning:", "").strip()
    return valence_rating, arousal_rating, alignment_score, reasoning

for example in emotion_examples:
    valence_ratings = []
    arousal_ratings = []
    alignment_scores = []
    reasonings = []
    for claim in tqdm(example.relevant_claims):
        valence_rating, arousal_rating, alignment_score, reasoning = calculate_expert_alignment_score(example.text, example.llm_label, claim)
        # if category is None:
        #     continue
        valence_ratings.append(valence_rating)
        arousal_ratings.append(arousal_rating)
        alignment_scores.append(alignment_score)
        reasonings.append(reasoning)
    example.valence_ratings = valence_ratings
    example.arousal_ratings = arousal_ratings
    example.alignment_scores = alignment_scores
    example.reasonings = reasonings
    
    break
    

100%|██████████| 2/2 [00:03<00:00,  1.74s/it]


In [25]:
print(emotion_examples[0].valence_ratings)
print(emotion_examples[0].arousal_ratings)
print(emotion_examples[0].alignment_scores)
print(emotion_examples[0].reasonings)

[1.5, 1.0]
[1.0, 0.5]
[1.5, 1.0]
['The claim describes an amused reaction, implied by the "tongue-in-cheek or playful" response, suggesting a positive emotional undercurrent. The term "Creepy" might be conventionally negative, but the context here is playful, resulting in a high positive valence. The arousal is moderately high due to the playful tone, indicating some level of energy and engagement, though it doesn\'t reach the intensity of a very high-arousal situation.', 'The claim suggests a mildly positive sentiment through the lens of amusement, as indicated by the use of "tbh" in a casual and informal manner. The use of "creepy" in combination with a light-hearted context like "Yoda voicepack" implies a non-threatening, playful kind of creepiness, contributing to a slightly positive valence. The arousal rating reflects the light-hearted, yet mildly engaging nature of the sentiment, but it\'s not particularly intense or energetic.']
